In [1]:
# Import libraries 
import pandas as pd
import subprocess

## Chartbook Minimal Example - US Wage Growth

Three parts:

1) Gather data, apply functions, store cleaned data
2) Convert numbers into text and generate data descriptions
3) Match the data and text with a LaTex (or other) template

Tools: Python, R, Stata or similar for parts (1) and (2). LaTeX (pdf) or web-based for part (3). In theory, AI (Gemini) could do part (2) as well. 

This example is written in python. If you are new to python, I suggest using 'miniconda'. 

The example shows a way to download wage data from FRED, apply calculations, and save the results. Next, these results are entered into a set of functions and script to generate two sentences of descriptive text. This text is saved locally as a .txt file. Finally, a separate LaTex (.tex) file is used as a template. The .tex file has a "hard-keyed" portion, a placeholder for the .txt file, and a placeholder chart waiting to read the .csv wage growth data. When the .tex file is compiled, it will generate/update a .pdf output.

#### Step 1: Gather data, apply functions, store cleaned data

Download wage data from FRED and calculate the three-month over three-month growth rate. Save the results locally as a .csv file.

In [2]:
# Download AHETPI series from FRED
url = 'https://fred.stlouisfed.org/data/AHETPI'
df = pd.read_html(url, parse_dates=True)[1].set_index('DATE')

In [3]:
# Show last 5 observations
df.tail()

,VALUE
DATE,
2024-10-01,30.49
2024-11-01,30.58
2024-12-01,30.67
2025-01-01,30.80
2025-02-01,30.89


In [4]:
# Calculate 3-month moving average
df['MA'] = df['VALUE'].rolling(3).mean()

In [5]:
df.tail()

,VALUE,MA
DATE,,
2024-10-01,30.49,30.380000
2024-11-01,30.58,30.483333
2024-12-01,30.67,30.580000
2025-01-01,30.80,30.683333
2025-02-01,30.89,30.786667


In [6]:
# Calculate 3M/3M change
df['CH3M'] = (((df['MA'] / df['MA'].shift(3))**4) - 1) * 100

In [7]:
df.tail()

,VALUE,MA,CH3M
DATE,,,
2024-10-01,30.49,30.380000,4.187919
2024-11-01,30.58,30.483333,4.219403
2024-12-01,30.67,30.580000,4.113957
2025-01-01,30.80,30.683333,4.054071
2025-02-01,30.89,30.786667,4.040123


In [8]:
# Save result to csv
df.loc['2017':].to_csv('wages.csv')

#### Step 2: Generate text

Two sentences: the curent monthly and 3-month average values; the 3M/3M growth rate.

In [9]:
# Open csv file
data = pd.read_csv('wages.csv', parse_dates=['DATE'], index_col='DATE')
# Format latest date
ltdt = data.index[-1].strftime('%B %Y')
# Format wages with dollar sign and two decimals
lt = data.map('\\${:.2f}'.format)

# Describe the change: If small, "virtually unchanged", else "increase/decrease of X percent"
chval = data['CH3M'].iloc[-1]
chdir = 'decrease' if chval < 0 else 'increase' # Direction of change
if abs(chval) <= 0.2:   # Small values
    chtxt = 'virtually no change'
else:
    chtxt = f'an annualized {chdir} of {abs(chval):.1f} percent'
    
text = (f'As of {ltdt}, the earnings of production and non-supervisory '+
        f'workers average {lt.VALUE.iloc[-1]} per hour. Over the past '+
        f'three months, earnings average {lt.MA.iloc[-1]} per hour, '+
        f'{chtxt} from the previous three months.')
with open('wages.txt', 'w') as text_file:
        text_file.write(text)
print(text)

As of February 2025, the earnings of production and non-supervisory workers average \$30.89 per hour. Over the past three months, earnings average \$30.79 per hour, an annualized increase of 4.0 percent from the previous three months.


#### Step 3: Update template
This block of code will compile the latex file chartbook_example.tex with the latest data and text generated above. If the result is successful, the process will return code "0". 

In [10]:
# Compile latex template with latest data to generate pdf file
runcb = subprocess.run(['latex', 'chartbook_example.tex'], stdout=subprocess.DEVNULL)
if runcb.returncode == 0:
    print('Success! PDF file generated.')
else:
    print(runcb.returncode)

0


#### Scaling up the process

To scale up this process, I've written a few custom functions. For example value_text() will generate text from a single numerical value, and has lots of options to handle different use cases. 

In [11]:
from utils import value_text # Custom function to handle economic statistics 

In [12]:
# Apply custom function to wage growth data
value_text(chval, 'increase_of', adj='annualized')

'an annualized increase of four percent'

In [13]:
# Basic output
value_text(chval)

'increased four percent'

In [14]:
# Negative values
value_text(-8.27748)

'decreased 8.3 percent'

In [15]:
# Options for text
value_text(87.7628, 'contribution')

'contributed 87.8 percent'

In [16]:
# More casual text
value_text(73.311, 'contribution', casual=True)

'added 73.3 percent'

In [17]:
# Format as currency
value_text(6282.84346, 'plain', ptype=None, dollar=True, digits=2)

'\\$6,282.84'